In [1]:
import json
from pprint import pprint

first = r"C:\Users\Jonathan Gupton\Dropbox\spirelogs\20190201003826_1548999484.run.json"
with open(first, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the loaded JSON data
pprint(data)

{'ascension_level': 0,
 'boss_relics': [{'not_picked': ['Sozu', 'Lizard Tail'],
                  'picked': "Philosopher's Stone"},
                 {'not_picked': ["Pandora's Box", 'Orrery'],
                  'picked': 'Ring of the Serpent'}],
 'build_version': '2019-01-23',
 'campfire_choices': [{'floor': 15, 'key': 'REST'},
                      {'data': 'Apotheosis', 'floor': 24, 'key': 'SMITH'},
                      {'floor': 32, 'key': 'REST'},
                      {'data': 'Envenom', 'floor': 41, 'key': 'SMITH'},
                      {'floor': 44, 'key': 'REST'},
                      {'floor': 49, 'key': 'REST'}],
 'campfire_rested': 4,
 'campfire_upgraded': 2,
 'card_choices': [{'floor': 1,
                   'not_picked': ['Endless Agony', 'Dash'],
                   'picked': 'Poisoned Stab'},
                  {'floor': 6,
                   'not_picked': ['PiercingWail',
                                  'All Out Attack',
                                  'Leg Sweep'],

In [11]:
import os
import zipfile
import io

def collect_json_files_in_directory(dir_path: str) -> list:
    json_files = []
    # Walk through the directory recursively.
    for root, _, files in os.walk(dir_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith('.json'):
                json_files.append(file_path)
            elif file.lower().endswith('.zip'):
                json_files.extend(collect_json_files_in_zip(file_path, file_path))
    return json_files

def collect_json_files_in_zip(zip_path: str, base: str) -> list:
    json_files = []
    try:
        with zipfile.ZipFile(zip_path, 'r') as z:
            json_files.extend(collect_json_in_zipobj(z, base))
    except zipfile.BadZipFile:
        # Skip invalid ZIP files.
        pass
    return json_files

def collect_json_in_zipobj(z: zipfile.ZipFile, base: str) -> list:
    json_files = []
    for name in z.namelist():
        if name.lower().endswith('.json'):
            # Use an exclamation mark to separate the zip file path and the inner file name.
            json_files.append(f'{base}!{name}')
        elif name.lower().endswith('.zip'):
            try:
                nested_bytes = z.read(name)
                with zipfile.ZipFile(io.BytesIO(nested_bytes)) as nested_zip:
                    json_files.extend(collect_json_in_zipobj(nested_zip, f'{base}!{name}'))
            except zipfile.BadZipFile:
                # Skip invalid nested ZIP files.
                continue
    return json_files


spirelogs_path = r'C:\Users\Jonathan Gupton\Dropbox\spirelogs'
all_json_files = collect_json_files_in_directory(spirelogs_path)
all_json_files = sorted(all_json_files)
print(type(all_json_files))
print(len(all_json_files))

<class 'list'>
463833


In [15]:
all_json_files[-1]


'C:\\Users\\Jonathan Gupton\\Dropbox\\spirelogs\\20200531204213_1590972022.run.json'

In [16]:
import os
import json
import zipfile
import io
from collections import defaultdict

def process_json_data(data, counts):
    # If the JSON data contains the key "character_chosen", update the counts.
    if "character_chosen" in data:
        counts[data["character_chosen"]] += 1

def process_json_file(file_path, counts):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        process_json_data(data, counts)
    except Exception:
        pass

def process_json_from_zipobj(z, base, counts):
    for name in z.namelist():
        if name.lower().endswith('.json'):
            try:
                file_bytes = z.read(name)
                data = json.loads(file_bytes.decode('utf-8'))
                process_json_data(data, counts)
            except Exception:
                continue
        elif name.lower().endswith('.zip'):
            try:
                nested_bytes = z.read(name)
                with zipfile.ZipFile(io.BytesIO(nested_bytes)) as nested_zip:
                    process_json_from_zipobj(nested_zip, f'{base}!{name}', counts)
            except Exception:
                continue

def process_json_from_zip(zip_path, counts):
    try:
        with zipfile.ZipFile(zip_path, 'r') as z:
            process_json_from_zipobj(z, zip_path, counts)
    except Exception:
        pass

def count_character_chosen(dir_path: str) -> dict:
    counts = defaultdict(int)
    for root, _, files in os.walk(dir_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith('.json'):
                process_json_file(file_path, counts)
            elif file.lower().endswith('.zip'):
                process_json_from_zip(file_path, counts)
    return dict(counts)

if __name__ == '__main__':
    result = count_character_chosen(spirelogs_path)
    print(result)

{'THE_SILENT': 132295, 'IRONCLAD': 174785, 'DEFECT': 142711, 'CHRONO_CLASS': 58, 'RUNESMITH_CLASS': 5, 'THE_ELEMENTALIST': 8, 'THE_ANIMATOR': 32, 'SLIMEBOUND': 116, 'THE_SCRIBE': 3, 'CLOCKWORK': 1, 'THE_CATGIRL': 1, 'MARISA': 67, 'BLACKBEARD_CLASS': 9, 'THE_CONSTRUCT_MOD': 22, 'CHEN': 11, 'BEAKED_THE_CULTIST': 12, 'Kiyohime': 5, 'THE_SERVANT': 11, 'DragonServant': 2, 'THE_SEEKER': 6, 'TD1MADAO_CLASS': 7, 'SNECKO': 4, 'HAKUMEN': 17, 'THE_SENSHI': 4, 'MYSTIC_CLASS': 4, 'TheValiant': 1, 'THE_BUGKNIGHT': 12, 'THE_POKER_PLAYER': 5, 'DRUID': 3, 'GUARDIAN': 5, 'ILLYA': 7, 'OPM': 4, 'ASTROLOGER': 12, 'GREMLIN': 3, 'WATCHER': 13092, 'DANTE': 1, 'THE_BLADEMASTER': 3, 'THE_DUELIST': 2, 'HORNYDRAGON': 2, 'THE_CURSED': 15, 'SAIYAJIN': 3, 'REMILIA': 1, 'AbForeigner': 22, 'JUDGE_CLASS': 3, 'WITCH': 6, 'MagaServantEnum': 1, 'BARD': 2, 'MAD_SCIENTIST': 9, 'THE_JUGGERNAUT': 1, 'NECROMANCER': 9, 'CHLOE': 3, 'FallenAngel': 8, 'Giant': 3, 'KomeijiSatori': 4, 'KOISHI': 15, 'THE_FLORIST': 3, 'THE_ARTIST': 1,

In [ ]:
A

In [7]:
1

370

In [8]:
4

709

In [17]:
f

NameError: name 'f' is not defined